In [ ]:
%load_ext autoreload
%autoreload 2

## Generating CIPHER input files

This notebook has some examples of generating CIPHER input files.

In [ ]:
from cipher_input import (
    CIPHERInput,
    InterfaceDefinition,
    PhaseTypeDefinition,
    MaterialDefinition,
)
from quats import quat_sample_random
from utilities import read_shockley

import plotly.express as px # we could use matplotlib here instead
import numpy as np

Set up some default material and interface properties, and solution parameters, for the purposes of generating valid input files. You will want to customise these to your problem.

In [ ]:
mat_props = {
    'chemicalenergy': 'none',
    'molarvolume': 1e-5,
    'temperature0': 500.0,
}

int_props_1 = {
    'energy': {'e0': 5e+8},
    'mobility': {'m0': 1e-11},
}
int_props_2 = {
    'energy': {'e0': 5e+8},
    'mobility': {'m0': 1e-11},
}

solution_params_1 = {
  'abstol': 0.0001,
  'amrinterval': 25,
  'initblocksize': [1, 1, 1],
  'initcoarsen': 6,
  'initrefine': 7,
  'interfacewidth': 4,
  'interpolation': 'cubic',
  'maxnrefine': 7,
  'minnrefine': 0,
  'outfile': 'out',
  'outputfreq': 100,
  'petscoptions': '-ts_adapt_monitor -ts_rk_type 2a',
  'random_seed': 1579993586,
  'reltol': 0.0001,
  'time': 100000000,
}

solution_params_2 = {**solution_params_1}
solution_params_2['initblocksize'] = [1, 1]

solution_params_3 = {**solution_params_2}
solution_params_3['initrefine'] = 6
solution_params_3['maxnrefine'] = 6

solution_params_4 = {**solution_params_1}
solution_params_4['initblocksize'] = [1, 64, 128]
solution_params_4['initrefine'] = 0
solution_params_4['initcoarsen'] = 0

### 1. Random Voronoi tessellation of phases

These example generate the geometry using a Voronoi tessellation of a set of random seed point.

#### Example 1.1: One interface per material-pair

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        properties=mat_props,
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_1_1 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=500,
    grid_size=[128, 128, 128],
    size=[128, 128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_1,
    interfaces=interfaces,
)

##### Write the input YAML file

In [ ]:
input_1_1.write_yaml("ex_1.1.yaml")

##### Visualise a slice of the phase map

In [ ]:
px.imshow(input_1_1.geometry.voxel_phase[20])

##### Visualise a slice of the phase interfaces (hiding bulk voxels)

In [ ]:
px.imshow(input_1_1.geometry.neighbour_voxels[20])

##### Visualise the interface map

This is the 2D symmetric matrix that CIPHER uses to assign each possible phase-pair to a given interface

In [ ]:
px.imshow(input_1_1.geometry.interface_map)

##### Visualise a slice of the interface indices of the interface voxels

In [ ]:
px.imshow(input_1_1.geometry.get_interface_idx()[20])

##### Visualise a slice of the material assignment

In [ ]:
px.imshow(input_1_1.geometry.voxel_material[20])

##### Visualise a slice of the phase-type assignment

In this case, this is identical to the phase-material assignment, because by default, one phase type will be applied to each material.

In [ ]:
px.imshow(input_1_1.geometry.voxel_phase_type[20])

##### Visualise in 3D - experimental (may crash!)

In [ ]:
input_1_1.geometry.show()

#### Example 1.2: Multiple interfaces types for a given phase-pair - equal distribution

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        target_volume_fraction=0.2,
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        target_volume_fraction=0.8,
        properties=mat_props,
    ),
]

# Define the interfaces:
# "low-angle" and "high-angle" will be equally distributed for the mat1-mat1 interfaces
interfaces=[
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='low-angle',
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='high-angle',
        properties=int_props_1,
    ),    
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_1_2 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=100,
    grid_size=[128, 128],
    size=[128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

In [ ]:
input_1_2.write_yaml("ex_1.2.yaml")

In [ ]:
px.imshow(input_1_2.geometry.voxel_phase)

In [ ]:
px.imshow(input_1_2.geometry.voxel_material)

In [ ]:
px.imshow(input_1_2.geometry.get_interface_idx())

#### Example 1.3: Multiple interfaces types for a given phase-pair - specified distribution

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        target_volume_fraction=0.9,
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        target_volume_fraction=0.1,
        properties=mat_props,
    ),
]

# Define the interfaces:
# "low-angle" and "high-angle" will be distributed according to `type_fraction`
interfaces=[
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='low-angle',
        type_fraction=0.7,
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='high-angle',
        type_fraction=0.3,
        properties=int_props_1,
    ),    
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_1_3 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=100,
    grid_size=[128, 128],
    size=[128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

input_1_3.write_yaml("ex_1.3.yaml")

In [ ]:
px.imshow(input_1_3.geometry.voxel_phase)

In [ ]:
px.imshow(input_1_3.geometry.voxel_material)

In [ ]:
px.imshow(input_1_3.geometry.get_interface_idx())

### 2. Voronoi tessellation of existing seed points

#### Example 2.1: using pre-existing seed positions for the Voronoi tessellation

For this, we just use `CIPHERInput.from_seed_voronoi` instead of `CIPHERInput.from_random_voronoi`, and pass `seeds` instead of `num_phases`, where `seeds` should be an `(N, 2)` or `(N, 3)` array for 2D or 3D, respectively. Seeds are specified in real-space units, so must be defined within `size`.

In [ ]:
# Here we define some seeds using CIPHERGeometry, but may define seeds in some other way.

from cipher_input import CIPHERGeometry

grid_size = [128, 128]
size = [128, 128]
seeds = CIPHERGeometry.get_unique_random_seeds(num_phases=50, grid_size=grid_size, size=size)

# visualise the seeds:
px.scatter(x=seeds[:, 0], y=seeds[:, 1])

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        target_volume_fraction=0.2,
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        target_volume_fraction=0.8,
        properties=mat_props,
    ),
]
# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_2_1 = CIPHERInput.from_seed_voronoi(
    materials=materials,    
    seeds=seeds,
    grid_size=grid_size,
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

input_2_1.write_yaml("ex_2.1.yaml")

In [ ]:
fig = px.imshow(input_2_1.geometry.voxel_phase)
fig.add_scatter(x=seeds[:, 0], y=seeds[:, 1], mode='markers')

### 3. Using pre-existing voxel-phase map

We can pass in directly the voxel map if we have it, using `CIPHERInput.from_voxel_phase_map`.

In [ ]:
# First let's generate a simple 2D voxel phase map. This could be generated in some other way.
from discrete_voronoi import DiscreteVoronoi

size = [64, 64]
num_phases = 10
voronoi_obj = DiscreteVoronoi.from_random(
    size=size,
    grid_size=[64, 64],
    num_regions=num_phases,
)
voxel_phase = voronoi_obj.region_ID

# visualise the voxel_phase map:
px.imshow(voxel_phase)

#### Example 3.1: phase-material assignment using specified volume fractions

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        target_volume_fraction=0.2,
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        target_volume_fraction=0.8,
        properties=mat_props,
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_3_1 = CIPHERInput.from_voxel_phase_map(
    voxel_phase=voxel_phase,
    materials=materials,
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_3,
    interfaces=interfaces,
)

input_3_1.write_yaml("ex_3.1.yaml")

In [ ]:
px.imshow(input_3_1.geometry.voxel_phase)

In [ ]:
px.imshow(input_3_1.geometry.voxel_material)

In [ ]:
# check actual volume fractions:
input_3_1.geometry.material_volume_fractions

#### Example 3.2: specify phase_material assignment as well

Instead of using `target_volume_fractions` within a `MaterialDefinition`, we can specify directly which phases are associated with each material:

In [ ]:
# Generate a phase material mapping. This could be done in some other way.
rng = np.random.default_rng()
phase_material = rng.choice(a=2, size=num_phases)
print(phase_material)

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        properties=mat_props,
        phases=np.where(phase_material == 0)[0],
    ),
    MaterialDefinition(
        name="mat2",
        properties=mat_props,
        phases=np.where(phase_material == 1)[0],
    ),
]

In [ ]:
# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_3_2 = CIPHERInput.from_voxel_phase_map(
    voxel_phase=voxel_phase,
    materials=materials,
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_3,
    interfaces=interfaces,
)

input_3_2.write_yaml("ex_3.2.yaml")

In [ ]:
px.imshow(input_3_2.geometry.voxel_phase)

In [ ]:
px.imshow(input_3_2.geometry.voxel_material)

### 4. Using pre-existing phase-interface maps

By default, if multiple interface definitions are provided for a material pair, the interfaces are randomly assigned in equal proportion, or according to the `type_fraction` parameter, if specified.

However, we can also provide the interface indices for a given set of phases manually. In the example below, we have two interface definitions for the material pair (`mat1`, `mat2`), one labelled `low-angle` and another labelled `high-angle`. We use the `phase_pairs` parameter in both of these interface definitions to select the subset of phase interfaces that should belong to each interface definition. Note that the union of this parameter across all interface definitions for a given material pair must be the set of all phase pairs belonging to that material pair.

Using pre-existing phase-interface maps only really makes sense if we are also using a pre-existing voxel-phase map and phase-material map, so let's generate a dummy voxel-phase map and dummy phase-material map:

#### Example 4.1

In [ ]:
# First let's generate a simple 2D voxel phase map. This could be generated in some other way.
from discrete_voronoi import DiscreteVoronoi

size = [64, 64]
num_phases = 10
voronoi_obj = DiscreteVoronoi.from_random(
    size=size,
    grid_size=[64, 64],
    num_regions=num_phases,
)
voxel_phase = voronoi_obj.region_ID

# visualise the voxel_phase map:
px.imshow(voxel_phase)

In [ ]:
# Generate a phase material mapping. This could be done in some other way.
rng = np.random.default_rng()
phase_material = rng.choice(a=2, size=num_phases)
print(phase_material)

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        properties=mat_props,
        phases=np.where(phase_material == 0)[0],
    ),
    MaterialDefinition(
        name="mat2",
        properties=mat_props,
        phases=np.where(phase_material == 1)[0],
    ),
]

In [ ]:
# We are concerned with the `mat1-mat1` interfaces (i.e. 0-0)
mat_A_idx = 0
mat_B_idx = 0

mat_A_phase_idx = np.where(phase_material == mat_A_idx)[0]
mat_B_phase_idx = np.where(phase_material == mat_B_idx)[0]

A_idx = np.repeat(mat_A_phase_idx, mat_B_phase_idx.shape[0])
B_idx = np.tile(mat_B_phase_idx, mat_A_phase_idx.shape[0])

map_idx = np.vstack((A_idx, B_idx))
map_idx_srt = np.sort(map_idx, axis=0)  # map onto upper triangle
map_idx_uniq = np.unique(map_idx_srt, axis=1)  # get unique pairs only

# remove diagonal elements (a phase can't have an interface with itself)
map_idx_non_trivial = map_idx_uniq[:, map_idx_uniq[0] != map_idx_uniq[1]]

# arbitrarily split up into type 1 and type 2:
type_1_phase_pairs, type_2_phase_pairs = np.array_split(map_idx_non_trivial, 2, axis=1)

In [ ]:
print(type_1_phase_pairs, '\n')
print(type_2_phase_pairs)

In [ ]:
# Define the interfaces:
interfaces=[
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='low-angle',
        phase_pairs=type_1_phase_pairs.T,
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='high-angle',
        phase_pairs=type_2_phase_pairs.T,
        properties=int_props_1,
    ),    
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties=int_props_1,
    ),
]

input_4_1 = CIPHERInput.from_voxel_phase_map(
    voxel_phase=voxel_phase,
    materials=materials,    
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_3,
    interfaces=interfaces,
)

In [ ]:
px.imshow(input_4_1.geometry.voxel_phase)

In [ ]:
px.imshow(input_4_1.geometry.get_interface_idx())

In [ ]:
input_4_1.write_yaml("ex_4.1.yaml")

### 5. Specifying multiple phase types within a material

#### Example 5.1

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        phase_types=[
            PhaseTypeDefinition(type_label='type_1', target_type_fraction=0.3),
            PhaseTypeDefinition(type_label='type_2', target_type_fraction=0.7),
        ],
        properties=mat_props,
    ),
]

# Define the interfaces (using `phase_types` instead of `materials`):
interfaces = [
    InterfaceDefinition(
        phase_types=("mat1-type_1", "mat1-type_2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        phase_types=("mat1-type_1", "mat1-type_1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        phase_types=("mat1-type_2", "mat1-type_2"),
        properties=int_props_1,
    ),
]

input_5_1 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=50,
    grid_size=[128, 128],
    size=[128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

In [ ]:
px.imshow(input_5_1.geometry.voxel_phase)

In [ ]:
px.imshow(input_5_1.geometry.voxel_material)

In [ ]:
px.imshow(input_5_1.geometry.voxel_phase_type)

In [ ]:
input_5_1.write_yaml("ex_5.1.yaml")

### 6. Importing from Dream.3D

We can build a synthetic microstructure in Dream3D and then generate a CIPHERInput from the Dream3D data (HDF5) file.

#### Example 6.1

In [ ]:
# Define the material properties (these are "phases" in Dream3D):
materials = [
    MaterialDefinition(
        name="Primary",
        properties=mat_props,
    ),
    MaterialDefinition(
        name="Precipitate",
        properties=mat_props,
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("Primary", "Primary"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("Precipitate", "Precipitate"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        materials=("Primary", "Precipitate"),
        properties=int_props_1,
    ),
]

input_6_1 = CIPHERInput.from_dream3D(
    path="example_data/dream3d/2D/synthetic_d3d.dream3d",
    materials=materials,
    interfaces=interfaces,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_4,
)

In [ ]:
input_6_1.write_yaml("ex_6.1.yaml")

In [ ]:
px.imshow(input_6_1.geometry.voxel_phase[0])

In [ ]:
px.imshow(input_6_1.geometry.voxel_material[0])

In [ ]:
# orientations are passed in from Dream3D as quaternions.
# here is the first quaternion component for each voxel:
px.imshow(input_6_1.geometry.voxel_orientation[0, ..., 0])

In [ ]:
input_6_1.geometry.show()

#### Example 6.2: Defining phase types within the same material via Dream3D

We can assign the different Dream3D "phases" to the same CIPHER "material" definition by using `phase_types`.

In [ ]:
# Define the material properties:
materials = [
    MaterialDefinition(
        name="mat1",
        phase_types = [
            PhaseTypeDefinition(type_label="phase_type_1"),
            PhaseTypeDefinition(type_label="phase_type_2"),
        ], 
        properties=mat_props,
    ),
]

# Define the interfaces, where we know specify phase_type pairs instead of material pairs:
interfaces = [
    InterfaceDefinition(
        phase_types=("mat1-phase_type_1", "mat1-phase_type_1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        phase_types=("mat1-phase_type_2", "mat1-phase_type_2"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        phase_types=("mat1-phase_type_1", "mat1-phase_type_2"),
        properties=int_props_1,
    ),
]

# We also need provide a `phase_type_map` to state which Dream3D phases correspond to which
# CIPHER phase types:
input_6_2 = CIPHERInput.from_dream3D(
    path="example_data/dream3d/2D/synthetic_d3d.dream3d",
    materials=materials,
    interfaces=interfaces,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_4,
    phase_type_map={
        'Primary': 'mat1-phase_type_1',
        'Precipitate': 'mat1-phase_type_2',
    }
)

In [ ]:
px.imshow(input_6_2.geometry.voxel_phase[0])

In [ ]:
px.imshow(input_6_2.geometry.voxel_material[0])

In [ ]:
px.imshow(input_6_2.geometry.voxel_phase_type[0])

### 7. Encoding texture information (via interface properties)

Within a given material, we might want to encode texture information. This can be done via interfacial properties, and by using `phase_types`. For example, if each CIPHER phase is assigned an orientation, then we could calculate a misorientation between phase pairs, and then use the empirical Read-Shockley relationship to provide grain boundary energies.

In [ ]:
RS_params = {
    'E_max': 1.2,
    'theta_max': 50,
    'degrees': True,
}
theta_deg = np.linspace(0.0, 100)
energy = read_shockley(theta_deg, **RS_params)
fig = px.line(
    x=theta_deg,
    y=energy,
    labels={"x": "Misorientation angle /deg.", "y": "GB energy / Jm^-2"},
    title='Read-Shockley relationship for LAGBs',
    width=600,
)
fig

#### Example 7.1: passing orientations directly

In [ ]:
# Define phase indices for each material:
phases_1 = [0, 1]
phases_2 = [2, 3]

# Define the material properties and pass in orientations:
materials = [
    MaterialDefinition(
        name="mat1",
        phase_types=[
            PhaseTypeDefinition(
                phases=phases_1,
                orientations=quat_sample_random(len(phases_1)),
            ),
        ],
        properties=mat_props,
    ),
    MaterialDefinition(
        name="mat2",
        phase_types=[
            PhaseTypeDefinition(
                phases=phases_2,
                orientations=quat_sample_random(len(phases_2)),
            ),
        ],
        properties=mat_props,
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        phase_types=("mat1", "mat1"),
        properties=int_props_1,
    ),
    InterfaceDefinition(
        phase_types=("mat2", "mat2"),
        properties=int_props_1,
    ),    
    InterfaceDefinition(
        phase_types=("mat1", "mat2"),
        properties=int_props_1,
    ),
]

input_7_1 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=len(phases_1) + len(phases_2),
    grid_size=[128, 128],
    size=[128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

In [ ]:
px.imshow(input_7_1.geometry.voxel_phase)

In [ ]:
# get orientations of all phases:
input_7_1.geometry.phase_orientation

In [ ]:
# show the first quaternion component of each voxel:
px.imshow(input_7_1.geometry.voxel_orientation[..., 0])

Now find the misorientation angles between all phase-pairs:

In [ ]:
misori = input_7_1.geometry.get_misorientation_matrix()
px.imshow(misori)

Now convert this misorientation into grain boundary energies via the Read-Shockley relationship:

In [ ]:
E_GB = read_shockley(misori, **RS_params)
px.imshow(E_GB)

We can then "expand" the existing interface definition into multiple interface definitions (one for each phase pair):

In [ ]:
input_7_1.apply_interface_property(
    base_interface_name="mat1-mat2",
    property_name=('energy', 'e0'),
    property_values=E_GB * 3e8,
)

Check the number of interfaces has increased. There should be:

`num_phases * (num_phases -1 ) / 2`

In [ ]:
int(input_7_1.geometry.num_phases * (input_7_1.geometry.num_phases - 1) / 2)

In [ ]:
len(input_7_1.geometry.interfaces)

In [ ]:
input_7_1.write_yaml('ex_7.1.yaml')

#### Example 7.2: passing orientations directly (binning interfaces by GB energy)

In [ ]:
RS_params = {
    'E_max': 1.2,
    'theta_max': 50,
    'degrees': True,
}
theta_deg = np.linspace(0, 100)
energy = read_shockley(theta_deg, **RS_params)
fig = px.line(
    x=theta_deg,
    y=energy,
    labels={"x": "Misorientation angle /deg.", "y": "GB energy / Jm^-2"},
    title='Read-Shockley relationship for LAGBs',
    width=600,
)
fig

In [ ]:
num_phases = 10

# Define the material properties and pass in orientations:
materials = [
    MaterialDefinition(
        name="mat1",
        phase_types=[
            PhaseTypeDefinition(
                phases=np.arange(num_phases),
                orientations=quat_sample_random(num_phases)
            ),
        ],
        properties=mat_props,
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        phase_types=("mat1", "mat1"),
        properties=int_props_1,
    ),
]

input_7_2 = CIPHERInput.from_random_voronoi(
    materials=materials,
    num_phases=num_phases,
    grid_size=[128, 128],
    size=[128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_params_2,
    interfaces=interfaces,
)

misori = input_7_2.geometry.get_misorientation_matrix()
E_GB = read_shockley(misori, **RS_params)

input_7_2.apply_interface_property(
    base_interface_name="mat1-mat1",
    property_name=('energy', 'e0'),
    property_values=E_GB * 3e8,
    additional_metadata={'misorientation': misori},
    bin_edges=np.linspace(0, RS_params['E_max'] * 3e8, num=30),
)
input_7_2.write_yaml('ex_7.2.yaml')